In [1]:
import pandas as pd

In [4]:
import tensorflow_hub as hub
import tensorflow as tf
import os

Create a class to keep each sentence tokenized (with and without stop words)

In [ ]:
import nltk

STOP = set(nltk.corpus.stopwords.words("english"))

class Sentence:
    
    def __init__(self, sentence):
        self.raw = sentence
#         normalized_sentence = sentence.replace("‘", "'").replace("’", "'")
        self.tokens = [t.lower() for t in nltk.word_tokenize(sentence)]
        self.tokens_without_stop = [t for t in self.tokens if t not in STOP]

Clean and format each phrase

In [ ]:
df_hc.keyword = df_hc.keyword.apply(lambda x: re.sub(r'[\/]',' ',x))
df_hc.keyword = df_hc.keyword.apply(lambda x: re.sub(r'[-]',' ',x))
df_hc.keyword = df_hc.keyword.apply(lambda x: re.sub(r'[\s]+',' ',x))
df_hc[df_hc.keyword == '']
df_hc['sents'] = df_hc.keyword.apply(lambda x: ' '.join(Sentence(x).tokens_without_stop))

We use tensorflow hub to create sentence embeddings

In [ ]:
embed = hub.Module('https://tfhub.dev/google/universal-sentence-encoder-large/3')

In [ ]:
embeddings = []
with tf.Session() as session:
    session.run([tf.global_variables_initializer(), tf.tables_initializer()])
    embeddings = session.run(embed(df_hc.sents.tolist()))

In [ ]:
df_hc['embedding'] = None
for i in list(df_hc.index):
    df_hc.set_value(index=i, col='embedding', value=embeddings[i])
df_hc.head()

In [140]:
# a = df_embed.sample(20)
# a

In [139]:
# df_hc = a

In [138]:
# df_ = pd.DataFrame.from_items(zip(df_hc.embedding.index, df_hc.embedding.values)).T
# df_.head()

We reduce the 512 dimensional vectors to 2 dimensions for cartesian visualization.

In [123]:
from sklearn.manifold import TSNE
import numpy as np
tsne_model_en_2d = TSNE(perplexity=30, n_components=2, init='pca', n_iter=3500, random_state=32)
# embeddings_en_2d = np.array(tsne_model_en_2d.fit_transform(df_))
embeddings_en_2d = np.array(tsne_model_en_2d.fit_transform(embeddings))
df_hc['x1'] = embeddings_en_2d[:, 0]
df_hc['y1'] = embeddings_en_2d[:, 1]

In [124]:
df_hc[['sents', 'embedding', 'x1', 'y1']].head()

,sents,embedding,x1,y1
3659,hypertension,"[-0.02381404, 0.01688445, -0.027617605, -0.032...",-268.441681,-632.560730
2254,babesiosis,"[-0.023700295, 0.004752079, 0.020919636, -0.01...",-471.195404,683.157593
2708,orbital surgery,"[0.0625993, 0.013124733, -0.0042589367, -0.017...",-72.664268,432.296814
1608,pimples,"[-0.0044431803, -0.003657779, -0.0081782, -0.0...",-165.546829,43.825447
3484,alopecia areata treatment,"[0.0294933, 0.017332012, -0.006409007, 0.00635...",-15.864203,-297.577911


In [125]:
df_hc.reset_index(drop=True, inplace=True)
df_hc = df_hc[['sents', 'embedding', 'x1', 'y1']]
df_hc.head()

,sents,embedding,x1,y1
0,hypertension,"[-0.02381404, 0.01688445, -0.027617605, -0.032...",-268.441681,-632.560730
1,babesiosis,"[-0.023700295, 0.004752079, 0.020919636, -0.01...",-471.195404,683.157593
2,orbital surgery,"[0.0625993, 0.013124733, -0.0042589367, -0.017...",-72.664268,432.296814
3,pimples,"[-0.0044431803, -0.003657779, -0.0081782, -0.0...",-165.546829,43.825447
4,alopecia areata treatment,"[0.0294933, 0.017332012, -0.006409007, 0.00635...",-15.864203,-297.577911


We compute cosine similarity between the vectors returning a 20*20 matrix

In [126]:
from sklearn.metrics import pairwise_distances
distance = pairwise_distances(X=df_, Y=df_, metric='cosine')

In [127]:
distance = 1 - distance

In [134]:
data = []
for i, row in df_hc.iterrows():
    data.append({'count': 1,
                 'text': row['sents'],
                 'id': i,
                 'x': row['x1'],
                 'y': row['y1'],
                 'similar': []})
    for ix, r in df_hc[df_hc.sents != row['sents']].iterrows():
        data[-1]['similar'].append({'count': 1,
                                    'text': r['sents'],
                                    'id': ix,
                                    'score': round(distance[i][ix], 2)})

In [137]:
import json
with open('static/js/data.json', 'w') as outfile:
    json.dump(data, outfile)